# Hill climb over TSP problem

In [1]:
#!pip install tsplib95

  Using cached tabulate-0.8.7-py3-none-any.whl (24 kB)


In [109]:
# Imposta as bibliotecas necessárias
#import geopy.distance
import tsplib95
import pandas as pd
import time
from random import randint

In [134]:
# Calcula o custo de viagem entre duas cidades - usando a distância geodésica
def costFunction(coords_1, coords_2, units='km'):
    if (units == 'km'):
        return geopy.distance.distance(coords_1, coords_2).km
    elif (units == 'miles'):
        return geopy.distance.distance(coords_1, coords_2).miles
# Example
#print(costFunction((52.2296756, 21.0122287), (52.406374, 16.9251681)))

# Calcula o custo total do problema
def totalCost(problem, nodes_df):
    cost = 0
    for row in nodes_df.itertuples(): #head(n=6).itertuples():
        # Testa se é o último nó
        if (row.Index < nodes_df.size-1):
            actualNode = row.NodeId
            nextNode = nodes_df.loc[row.Index+1].NodeId
            
            # Calcula a distância entre o nó atual e o próximo nó
            cost += problem.get_weight(actualNode, nextNode)
            #cost += costFunction((nodes_df.loc[row.Index].Latitude, nodes_df.loc[row.Index].Longitude), 
            #                     (nodes_df.loc[row.Index+1].Latitude, nodes_df.loc[row.Index+1].Longitude))
    return cost

# Calcula o custo para uma região contendo 4 nós em sequência
def areaCost(problem, nodes_df, i):
    cost = 0
    
    if (i > 0):
        cost += problem.get_weight(nodes_df.iat[i-1,0], nodes_df.iat[i,0])
    
    if (i < nodes_df.size-1):
        cost += problem.get_weight(nodes_df.iat[i,0], nodes_df.iat[i+1,0])
    
    return cost

# Executa o algoritmo Hill Climb para um problema de TSP, cuja ordem de visitação dos nós está indicada em um dataframe 
def hillClimb(problem, nodes_df, iterations=0):
    # Calcula o custo da configuração incial do problema
    cost_zero = totalCost(problem, nodes_df)
    print("Custo inicial = " + str(cost_zero))

    # Armazena o melhor custo e a melhor configuração até o momento
    cost_best = cost_zero
    nodes_df_best = nodes_df.copy()

    # Enquanto o custo total estiver diminuindo, testa as diferentes permutas entre os nós (máximo de 10 iterações)
    flgExit = False
    iteration = 1
    print('Testando as permutações entre os nós do problema')
    while (not flgExit) and (iteration != iterations):
        print('Iteração ' + str(iteration), end='\r')
        
        # Armazena a melhor diferença de custo entre as permutas
        cost_dif = 0    
            
        # Permuta todos os nós adjacentes e verifica se o custo da configuração melhorou
        for i, row in nodes_df.itertuples():
        #for i in range(0, nodes_df.size-1):
            # Cria uma cópia do dataframe na configuração atual
            nodes_df_temp = nodes_df.copy()

            if (i < nodes_df.size-1):
                # Calcula o custo na região dos elementos i e i+1
                cost_temp1 = areaCost(problem, nodes_df_temp, i)
                cost_temp1 += areaCost(problem, nodes_df_temp, i+1)

                # Permuta os valores nas posições i e i+1
                a, b = nodes_df_temp.iat[i,0], nodes_df_temp.iat[i+1,0]
                nodes_df_temp.iat[i,0], nodes_df_temp.iat[i+1,0] = b, a

                # Verifica se o custo na região diminuiu após a permuta e guarda os valores para a maior redução de custo
                cost_temp2 = areaCost(problem, nodes_df_temp, i)
                cost_temp2 += areaCost(problem, nodes_df_temp, i+1)
                if (cost_dif < (cost_temp1 - cost_temp2)):
                    cost_dif = cost_temp1 - cost_temp2
                    nodes_df_best = nodes_df_temp.copy()

        # Verifica se o custo do melhor filho superou o do pai
        if (cost_dif > 0):
            #print('Melhorou: ' + str(cost_dif))
            cost_best -= cost_dif
            nodes_df = nodes_df_best.copy()
            flgExit = False
        else:
            #print('Piorou')
            flgExit = True

        iteration += 1
    
    cost_best = totalCost(problem, nodes_df_best)
    print('')
    print("Custo final = " + str(cost_best))
    print("Diferença = " + str(cost_zero - cost_best))
    print('Iterações = ' + str(iteration))
    
    return cost_best, nodes_df_best, iteration

# Executa o algoritmo Hill Climb para um problema de TSP, cuja ordem de visitação dos nós está indicada em um dataframe 
def hillClimbTB(problem, nodes_df, iterations=0, backward=False, random=False):
    # Calcula o custo da configuração incial do problema
    cost_zero = totalCost(problem, nodes_df)
    print("Custo inicial = " + str(cost_zero))

    # Armazena o melhor custo e a melhor configuração até o momento
    cost_best = cost_zero
    nodes_df_best = nodes_df.copy()

    # Enquanto o custo total estiver diminuindo, testa as diferentes permutas entre os nós (máximo de 10 iterações)
    flgExit = False
    iteration = 1
    print('Testando as permutações entre os nós do problema')
    while (not flgExit):
        print('Iteração ' + str(iteration), end='\r')
        #print('Iteração ' + str(iteration))
        
        # Armazena a melhor diferença de custo entre as permutas
        cost_dif = 0    
            
        # Permuta todos os nós adjacentes quando a permutação for vantajosa
        for i, row in nodes_df.itertuples():
        #for i in range(0, nodes_df.size-1):
            # FORWARD STEP
            # Cria uma cópia do dataframe na configuração atual
            nodes_df_temp = nodes_df_best.copy()
            
            # Calcula o custo na região dos elementos i e i+1
            if (i < nodes_df.size-1):
                cost_temp1 = areaCost(problem, nodes_df_temp, i)
                cost_temp1 += areaCost(problem, nodes_df_temp, i+1)

                # Permuta os valores nas posições i e i+1
                a, b = nodes_df_temp.iat[i,0], nodes_df_temp.iat[i+1,0]
                nodes_df_temp.iat[i,0], nodes_df_temp.iat[i+1,0] = b, a

                # Verifica se o custo na região diminuiu após a permuta e guarda os valores para continuar trabalhando com a melhor configuração até o momento 
                cost_temp2 = areaCost(problem, nodes_df_temp, i)
                cost_temp2 += areaCost(problem, nodes_df_temp, i+1)
                if ((cost_temp1 - cost_temp2) > 0):
                    cost_dif += cost_temp1 - cost_temp2
                    nodes_df_best = nodes_df_temp.copy()
            #print('Custo forward: ' + str(cost_dif))
                
            # BACKWARD STEP
            if (backward):
                # Cria uma cópia do dataframe na configuração atual
                nodes_df_temp = nodes_df_best.copy()

                # Pega um elemento do final
                n = (nodes_df_temp.size - 1) - i
                    
                if (n > 1):
                    # Calcula o custo na região dos elementos n-2 e n
                    cost_temp1 = areaCost(problem, nodes_df_temp, n-2)
                    cost_temp1 += areaCost(problem, nodes_df_temp, n)

                    # Permuta os valores nas posições n-2 e n
                    a, b = nodes_df_temp.iat[n-2,0], nodes_df_temp.iat[n,0]
                    nodes_df_temp.iat[n-2,0], nodes_df_temp.iat[n,0] = b, a

                    # Verifica se o custo na região diminuiu após a permuta e guarda os valores para continuar trabalhando com a melhor configuração até o momento 
                    cost_temp2 = areaCost(problem, nodes_df_temp, n-2)
                    cost_temp2 += areaCost(problem, nodes_df_temp, n)
                    if ((cost_temp1 - cost_temp2) > 0):
                        cost_dif += cost_temp1 - cost_temp2
                        nodes_df_best = nodes_df_temp.copy()
                #print('Custo backward: ' + str(cost_dif))
            
            # RANDOM STEP
            if (random):
                # Cria uma cópia do dataframe na configuração atual
                nodes_df_temp = nodes_df_best.copy()
                
                # Pega um elemento de uma posição aleatória
                v = randint(0, nodes_df_temp.size - 1)

                # Calcula o custo na região dos elementos nas posições i e v antes da permuta
                cost_temp1 = areaCost(problem, nodes_df_temp, i)
                cost_temp1 += areaCost(problem, nodes_df_temp, v)

                # Permuta os valores nas posições i e v
                a, b = nodes_df_temp.iat[i,0], nodes_df_temp.iat[v,0]
                nodes_df_temp.iat[i,0], nodes_df_temp.iat[v,0] = b, a

                # Calcula o custo na região dos elementos nas posições i e v depois da permuta
                cost_temp2 = areaCost(problem, nodes_df_temp, i)
                cost_temp2 += areaCost(problem, nodes_df_temp, v)
                
                if ((cost_temp1 - cost_temp2) > 0):
                    cost_dif += cost_temp1 - cost_temp2
                    nodes_df_best = nodes_df_temp.copy()
                    
        # CRITÉRIOS DE PARADA
        # Verifica se as permutações ainda estão diminuindo o custo do problema
        if (cost_dif > 0):  
            #print('Melhorou: ' + str(cost_dif))
            nodes_df = nodes_df_best.copy()
            flgExit = False
            
             # Verifica se atingiu o limite de iterações
            if (iterations == 0) or (iteration < iterations):
                iteration += 1
            else:
                flgExit = True
        else:
            #print('Piorou')
            flgExit = True
    
    cost_best = totalCost(problem, nodes_df_best)
    
    print('')
    print("Custo final = " + str(cost_best))
    print("Diferença = " + str(cost_zero - cost_best))
    print('Iterações = ' + str(iteration))
    
    return cost_best, nodes_df_best, iteration

In [162]:
# Carrega o dataset do TSP
#dataset = 'DjiboutiTSP_38'
dataset = 'UruguayTSP_734'
#dataset = 'IrelandTSP_8246'

problem = tsplib95.load(dataset + '.tsp')

count = 1
while count <= 10:
    # Gera uma configuração aleatória do problema
    print("Gerando configuração aleatória")
    nodes_df = pd.DataFrame(list(problem.get_nodes()), columns = ['NodeId'])
    nodes_df = nodes_df.sample(frac=1).reset_index(drop=True)

    # Executa o Hill Climb
    print("### HillClim Puro ###")
    begin = time.time()
    cost_best, nodes_df_best, iteration = hillClimb(problem, nodes_df)
    end = time.time()
    print(end - begin)
    # Salva um arquivo com a melhor configuração encontrada
    nodes_df_best.to_csv('Results - ' + dataset + '\\' + 'HillClimb Puro' +'\\' + 'Nota 2 - HillClimb Puro - ' + dataset + ' - ' + str(count) + '.csv')
    
    # Executa o Hill Climb TB
    print("### HillClim TB ###")
    begin = time.time()
    cost_best, nodes_df_best, iteration = hillClimbTB(problem, nodes_df, iterations=0, backward=True, random=True)
    end = time.time()
    print(end - begin)
    # Salva um arquivo com a melhor configuração encontrada
    nodes_df_best.to_csv('Results - ' + dataset + '\\' + 'HillClimb TB' +'\\' + 'Nota 2 - HillClimb TB - ' + dataset + ' - ' + str(count) + '.csv')
    
    count += 1
    
print('Fim!')

Gerando configuração aleatória
### HillClim Puro ###
Custo inicial = 1618676
Testando as permutações entre os nós do problema
Iteração 298
Custo final = 1235965
Diferença = 382711
Iterações = 299
76.85314846038818
### HillClim TB ###
Custo inicial = 1618676
Testando as permutações entre os nós do problema
Iteração 128
Custo final = 425446
Diferença = 1193230
Iterações = 128
115.11589574813843
Gerando configuração aleatória
### HillClim Puro ###
Custo inicial = 1630564
Testando as permutações entre os nós do problema
Iteração 305
Custo final = 1242788
Diferença = 387776
Iterações = 306
97.71046257019043
### HillClim TB ###
Custo inicial = 1630564
Testando as permutações entre os nós do problema
Iteração 149
Custo final = 443121
Diferença = 1187443
Iterações = 149
153.73380398750305
Gerando configuração aleatória
### HillClim Puro ###
Custo inicial = 1646970
Testando as permutações entre os nós do problema
Iteração 304
Custo final = 1242652
Diferença = 404318
Iterações = 305
101.86267685